In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
import re
import networkx as nx
import nltk
nltk.download(['punkt','stopwords'])
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
# from nltk.corpus import stopwords
# from nltk import bigrams

# import warnings
# warnings.filterwarnings("ignore")

# sns.set(font_scale=1.5)
# sns.set_style("whitegrid")
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# analyzer = SentimentIntensityAnalyzer()%matplotlib inline
# %config InlineBackend.figure_format = 'retina'

[nltk_data] Downloading package punkt to /Users/ellie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ellie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [85]:
dataset_df = pd.read_csv('dataset_to_train.csv')

dataset_df = dataset_df.drop_duplicates()
dataset_df

,Unnamed: 0,Unnamed: 0.1,tweet,class,clean_tweet,text
0,0,0,@kooawr watching amas :)),0,@kooawr watching amas :)),@kooawr watching amas :))
1,1,1,still can't wrap my head around why this zoo c...,0,still can't wrap head around zoo close i live ...,still can't wrap head around zoo close I live ...
2,2,2,SAVE YOUR TEARS FOR ANOTHER DAY,0,save your tears for another day,SAVE YOUR TEARS FOR ANOTHER DAY
3,3,3,Good day!!! #surfing #somewhere #goodwave #en...,0,good day!!! #surfing #somewhere #goodwave #enj...,Good day!!! #surfing #somewhere #goodwave #enj...
4,4,4,@poppingin50 have a nice day yh,0,@poppingin50 nice day yh,@poppingin50 nice day yh
...,...,...,...,...,...,...
1395,1395,95,But in the meantime...? Our basic needs are be...,1,but meantime...? our basic needs met. we're st...,But meantime...? Our basic needs MET. We're st...
1396,1396,96,@voltruto kaya pala trending ka always ning HH...,1,@voltruto kaya pala trending ka always ning hh...,@voltruto kaya pala trending ka always ning HH...
1397,1397,97,the more illegal the better i always say its hot,1,illegal better always say hot,illegal better always say hot
1398,1398,98,@simplysarris i feel bad even if i mute someon...,1,@simplysarris feel bad even mute someone 🙃 alw...,@simplysarris feel bad even mute someone 🙃 alw...


In [86]:
dataset_df['clean_tweet'] = dataset_df['tweet'].apply(lambda x: ' '.join([item for item in x.split() if item not in stopwords]))

In [87]:
dataset_df

,Unnamed: 0,Unnamed: 0.1,tweet,class,clean_tweet,text
0,0,0,@kooawr watching amas :)),0,@kooawr watching amas :)),@kooawr watching amas :))
1,1,1,still can't wrap my head around why this zoo c...,0,still can't wrap head around zoo close I live ...,still can't wrap head around zoo close I live ...
2,2,2,SAVE YOUR TEARS FOR ANOTHER DAY,0,SAVE YOUR TEARS FOR ANOTHER DAY,SAVE YOUR TEARS FOR ANOTHER DAY
3,3,3,Good day!!! #surfing #somewhere #goodwave #en...,0,Good day!!! #surfing #somewhere #goodwave #enj...,Good day!!! #surfing #somewhere #goodwave #enj...
4,4,4,@poppingin50 have a nice day yh,0,@poppingin50 nice day yh,@poppingin50 nice day yh
...,...,...,...,...,...,...
1395,1395,95,But in the meantime...? Our basic needs are be...,1,But meantime...? Our basic needs MET. We're st...,But meantime...? Our basic needs MET. We're st...
1396,1396,96,@voltruto kaya pala trending ka always ning HH...,1,@voltruto kaya pala trending ka always ning HH...,@voltruto kaya pala trending ka always ning HH...
1397,1397,97,the more illegal the better i always say its hot,1,illegal better always say hot,illegal better always say hot
1398,1398,98,@simplysarris i feel bad even if i mute someon...,1,@simplysarris feel bad even mute someone 🙃 alw...,@simplysarris feel bad even mute someone 🙃 alw...


In [88]:
dataset_df['text'] = dataset_df['clean_tweet']
dataset_df['text']

0                               @kooawr watching amas :))
1       still can't wrap head around zoo close I live ...
2                         SAVE YOUR TEARS FOR ANOTHER DAY
3       Good day!!! #surfing #somewhere #goodwave #enj...
4                                @poppingin50 nice day yh
                              ...                        
1395    But meantime...? Our basic needs MET. We're st...
1396    @voltruto kaya pala trending ka always ning HH...
1397                        illegal better always say hot
1398    @simplysarris feel bad even mute someone 🙃 alw...
1399    @dyutka LMFAO SAME OMG used go one jamaica itt...
Name: text, Length: 1400, dtype: object

In [89]:
def remove_url(txt):
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())
all_tweets_no_urls = [remove_url(tweet) for tweet in dataset_df['text']]
all_tweets_no_urls[:5]

['kooawr watching amas',
 'still cant wrap head around zoo close I live phantom cave dude playing him hed appear certain times day went get stamp hand ring',
 'SAVE YOUR TEARS FOR ANOTHER DAY',
 'Good day surfing somewhere goodwave enjoy Timmys Cafe',
 'poppingin50 nice day yh']

In [90]:
#lower_case = [word.lower() for word in df_new['text']]
dataset_df["clean_tweet"]=dataset_df["clean_tweet"].str.lower()
dataset_df['clean_tweet'] = dataset_df['clean_tweet'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

In [97]:
(dataset_df.isnull().any().any())
## No null values in our dataset- can ignore removing
dataset_df.info(null_counts=True)
df_new = dataset_df[dataset_df['clean_tweet'].notnull()]
df_new.info(null_counts=True)
df_new.isnull().any().any()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1400 entries, 0 to 1399
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    1400 non-null   int64 
 1   Unnamed: 0.1  1400 non-null   int64 
 2   tweet         1400 non-null   object
 3   class         1400 non-null   int64 
 4   clean_tweet   1400 non-null   object
 5   text          1400 non-null   object
dtypes: int64(3), object(3)
memory usage: 76.6+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1400 entries, 0 to 1399
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    1400 non-null   int64 
 1   Unnamed: 0.1  1400 non-null   int64 
 2   tweet         1400 non-null   object
 3   class         1400 non-null   int64 
 4   clean_tweet   1400 non-null   object
 5   text          1400 non-null   object
dtypes: int64(3), object(3)
memory usage: 76.6+ KB


False

In [100]:
users = df_new['clean_tweet'].str.startswith('@')
#dataset_df['clean_tweet'] = dataset_df['clean_tweet'].str.strip('@') 
#dataset_df
users
df_new=df_new[~users].reset_index(drop=True)

In [104]:
df_new

df_new.to_csv(r'/Users/ellie/Desktop/dataset_to_train.csv')

In [105]:
df_new

,Unnamed: 0,Unnamed: 0.1,tweet,class,clean_tweet,text
0,1,1,still can't wrap my head around why this zoo c...,0,still can't wrap head around zoo close i live ...,still can't wrap head around zoo close I live ...
1,2,2,SAVE YOUR TEARS FOR ANOTHER DAY,0,save your tears for another day,SAVE YOUR TEARS FOR ANOTHER DAY
2,3,3,Good day!!! #surfing #somewhere #goodwave #en...,0,good day!!! #surfing #somewhere #goodwave #enj...,Good day!!! #surfing #somewhere #goodwave #enj...
3,6,6,Uri Hyeop singing Spring Day - BTS Gosh! I nee...,0,uri hyeop singing spring day - bts gosh! i nee...,Uri Hyeop singing Spring Day - BTS Gosh! I nee...
4,7,7,Today is just be mean to kleo day I see.,0,today mean kleo day i see.,Today mean kleo day I see.
...,...,...,...,...,...,...
781,1386,86,Thank you @Hersheys for always having this com...,1,thank @hersheys always commercial! it’s favori...,Thank @Hersheys always commercial! It’s FAVORI...
782,1389,89,My heart. ♥ I will love him until the end of.....,1,my heart. ♥ i love end of... i always love him.,My heart. ♥ I love end of... I always love him.
783,1393,93,Deathstroke would have voted for him in 2016 a...,1,deathstroke would voted 2016 2020. he'd jump s...,Deathstroke would voted 2016 2020. He'd jump s...
784,1395,95,But in the meantime...? Our basic needs are be...,1,but meantime...? our basic needs met. we're st...,But meantime...? Our basic needs MET. We're st...
